In [1]:
import pandas
import matplotlib as mpl
import xarray as xr
import numpy as np
import datetime as dt
dir_data='F:/data/model_data/CMEM/global-reanalysis-phy-001-030-monthly/'
dir_data_clim='F:/data/model_data/CMEM/global-reanalysis-phy-001-030-monthly/clim/'
def sss_filename(d):
    dir_data='F:/data/model_data/CMEM/global-reanalysis-phy-001-030-monthly/'
    syr=str(d.year).zfill(4)
    smon=str(d.month).zfill(2)
    fname_tem=syr + '/'+ 'mercatorglorys12v1_gl12_mean_' + syr + smon + '.nc'
    filename = dir_data + fname_tem
    return filename

#get 0.25 km grid from cmc data to downsamplt this
filename = 'https://podaac-opendap.jpl.nasa.gov/opendap/allData/ghrsst/data/GDS2/L4/GLOB/CMC/CMC0.2deg/v2/1994/002/19940102120000-CMC-L4_GHRSST-SSTfnd-CMC0.2deg-GLOB-v02.0-fv02.0.nc'
ds_v2 = xr.open_dataset(filename)
new_lat = np.linspace(ds_v2.lat[0], ds_v2.lat[-1], ds_v2.dims['lat'])
new_lon = np.linspace(ds_v2.lon[0], ds_v2.lon[-1], ds_v2.dims['lon'])
ds_v2.close()

In [ ]:
#make cmc monthly average ssts
for lyr in range(1993,2017): #2017):
    ds_mnth=[]
    for imon in range(1,13):
        d = dt.date(lyr,imon,1) 
        filename = sss_filename(d)
        ds = xr.open_dataset(filename,drop_variables = ['usi','vsi','mlotst','zos','bottomT','sithick','siconc','thetao','uo','vo'])
        ds_subset = ds.sel(depth = ds.depth[0].data)
        ds_low_res = ds_subset.interp(latitude = new_lat,longitude = new_lon)
        ds.close()
        ds_mnth.append(ds_low_res)
    combined = xr.concat(ds_mnth, dim='time')
    fname_tem='clim/monthly_average_' + syr + 'mercatorglorys12v1_gl12_mean.nc'
    cmc_filename_out = dir_data + fname_tem
    combined.to_netcdf(cmc_filename_out)
    

In [ ]:
ds_low_res.so[0,0,0].data